# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [7]:
# your code here
from google.colab import files
import pandas as pd

uploaded = files.upload()

df = pd.read_csv('tic-tac-toe.csv')


Saving tic-tac-toe.csv to tic-tac-toe.csv


In [8]:
df.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


In [12]:
# your code here
# Make a copy to keep original safe
df_num = df.copy()

# Mapping
mapping = {'x': 1, 'o': -1, 'b': 0}

# Apply to all 9 grid columns
for col in ['TL','TM','TR','ML','MM','MR','BL','BM','BR']:
    df_num[col] = df_num[col].map(mapping)

# Map class column to binary (True → 1, False → 0)
df_num['class'] = df_num['class'].map({True: 1, False: 0})

df_num.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,1,1,1,1,-1,-1,1,-1,-1,1
1,1,1,1,1,-1,-1,-1,1,-1,1
2,1,1,1,1,-1,-1,-1,-1,1,1
3,1,1,1,1,-1,-1,-1,0,0,1
4,1,1,1,1,-1,-1,0,-1,0,1


In [13]:
# Separate inputs and target
X = df_num.drop('class', axis=1)
y = df_num['class']

print("Input shape:", X.shape)
print("Target shape:", y.shape)

Input shape: (958, 9)
Target shape: (958,)


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [15]:
# your code here
import tensorflow as tf
import numpy as np

# Convert to numpy arrays
X_np = X.to_numpy().astype(np.float32)
y_np = y.to_numpy().astype(np.int32)

# Create tf.data.Dataset
dataset = tf.data.Dataset.from_tensor_slices((X_np, y_np))

# Shuffle and split
dataset = dataset.shuffle(buffer_size=len(X_np), seed=42, reshuffle_each_iteration=False)

train_size = int(0.8 * len(X_np))
train_ds = dataset.take(train_size)
test_ds  = dataset.skip(train_size)

In [16]:
from sklearn.preprocessing import StandardScaler

# Convert TensorFlow datasets back to numpy for scaling
X_train_np = np.array([x.numpy() for x, _ in train_ds])
y_train_np = np.array([y.numpy() for _, y in train_ds])

X_test_np  = np.array([x.numpy() for x, _ in test_ds])
y_test_np  = np.array([y.numpy() for _, y in test_ds])

# Fit scaler on train, transform both
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_np)
X_test_scaled  = scaler.transform(X_test_np)

In [17]:
# Recreate datasets with scaled values
train_ds = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_np)).batch(32).prefetch(tf.data.AUTOTUNE)
test_ds  = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test_np)).batch(32).prefetch(tf.data.AUTOTUNE)

In [22]:
# Build a Sequential model (ReLU hidden layers, Softmax output)
from tensorflow import keras
from tensorflow.keras import layers

# Build Sequential model (ReLU hidden layers, Softmax output)
n_features = X_train_scaled.shape[1]

model = keras.Sequential([
    layers.Input(shape=(n_features,)),
    layers.Dense(32, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(2, activation="softmax")   # 2 output classes: 0 (False) / 1 (True)
])


In [24]:
# Compile the model
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",  # y values are integers 0/1
    metrics=["accuracy"]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,442 (5.63 KB)

 Trainable params: 1,442 (5.63 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
# Train the model
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=100,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5952 - loss: 0.6787 - val_accuracy: 0.6562 - val_loss: 0.6238
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6405 - loss: 0.6207 - val_accuracy: 0.7031 - val_loss: 0.5937
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6815 - loss: 0.5861 - val_accuracy: 0.6979 - val_loss: 0.5714
Epoch 4/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7286 - loss: 0.5533 - val_accuracy: 0.7292 - val_loss: 0.5519
Epoch 5/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7459 - loss: 0.5303 - val_accuracy: 0.7135 - val_loss: 0.5334
Epoch 6/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7466 - loss: 0.5090 - val_accuracy: 0.7396 - val_loss: 0.5122
Epoch 7/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7883 - loss: 0.4827 - val_accuracy: 0.7708 - val_loss: 0.4886
Epoch 8/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8016 - loss: 0.4558 - val_accuracy: 0.7656 - v

In [28]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"Test accuracy: {test_acc:.4f} | Test loss: {test_loss:.4f}")

Test accuracy: 0.9844 | Test loss: 0.1206


In [30]:
# Save the trained model
model.save("tic-tac-toe.keras")

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [32]:
# your code here

# Load the saved model
model = tf.keras.models.load_model("tic-tac-toe.keras")

# Pick a few random rows from your test set
idx = np.random.choice(len(X_test_scaled), size=5, replace=False)
sample_X = X_test_scaled[idx]
sample_y_true = y_test_np[idx]

# Make predictions (softmax probabilities)
pred_probs = model.predict(sample_X)

# Get predicted class (0 or 1)
pred_classes = np.argmax(pred_probs, axis=1)

# Display results neatly
for i in range(len(idx)):
    print(f"Row {idx[i]}:")
    print(f"  True label: {sample_y_true[i]} | Predicted: {pred_classes[i]}")
    print(f"  Probabilities: {pred_probs[i]}")
    print("-"*50)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Row 151:
  True label: 0 | Predicted: 0
  Probabilities: [0.99452233 0.00547761]
--------------------------------------------------
Row 16:
  True label: 1 | Predicted: 1
  Probabilities: [0.07429362 0.9257064 ]
--------------------------------------------------
Row 48:
  True label: 1 | Predicted: 1
  Probabilities: [0.2443792 0.7556208]
--------------------------------------------------
Row 28:
  True label: 1 | Predicted: 1
  Probabilities: [0.01043276 0.9895672 ]
--------------------------------------------------
Row 163:
  True label: 1 | Predicted: 1
  Probabilities: [0.01731502 0.982685  ]
--------------------------------------------------


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [33]:
# your code here
tf.random.set_seed(42)

def build_model(lr=1e-3, units=(64, 64, 32), dropout=0.2, input_dim=9):
    model = keras.Sequential([layers.Input(shape=(input_dim,))])
    for u in units:
        model.add(layers.Dense(u, activation="relu", kernel_initializer="he_normal"))
    if dropout and dropout > 0:
        model.add(layers.Dropout(dropout))
    model.add(layers.Dense(2, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=12, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=6, min_lr=1e-5, verbose=0),
]

configs = [
    {"name":"baseline_more_epochs", "lr":1e-3,  "units":(64,64),     "dropout":0.2, "epochs":200},
    {"name":"wider_deeper_lr5e-4",  "lr":5e-4,  "units":(128,128,64),"dropout":0.2, "epochs":200},
    {"name":"deeper_lr1e-4",        "lr":1e-4,  "units":(128,64,32), "dropout":0.1, "epochs":250},
]

results = []
best = None
for cfg in configs:
    print(f"\n=== Training: {cfg['name']} ===")
    model = build_model(lr=cfg["lr"], units=cfg["units"], dropout=cfg["dropout"], input_dim=9)
    history = model.fit(
        train_ds,
        validation_data=test_ds,
        epochs=cfg["epochs"],
        callbacks=callbacks,
        verbose=0
    )
    test_loss, test_acc = model.evaluate(test_ds, verbose=0)
    results.append((cfg["name"], test_acc, test_loss))
    if (best is None) or (test_acc > best[1]) or (test_acc == best[1] and test_loss < best[2]):
        best = (cfg["name"], test_acc, test_loss, model)

print("\nResults (name, acc, loss):")
for r in results:
    print(f"{r[0]:22s}  acc={r[1]:.4f}  loss={r[2]:.4f}")

print(f"\nBest: {best[0]}  acc={best[1]:.4f}  loss={best[2]:.4f}")

# Save the best one
best_model = best[3]
best_model.save("tic-tac-toe.keras")


=== Training: baseline_more_epochs ===

=== Training: wider_deeper_lr5e-4 ===

=== Training: deeper_lr1e-4 ===

Results (name, acc, loss):
baseline_more_epochs    acc=0.9948  loss=0.0894
wider_deeper_lr5e-4     acc=0.9896  loss=0.0788
deeper_lr1e-4           acc=0.9792  loss=0.1435

Best: baseline_more_epochs  acc=0.9948  loss=0.0894


**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here
'''
The first configuration (baseline_more_epochs) gave the best results:
accuracy = 0.9948, loss = 0.0894.

That improvement mainly came from:

* Training for more epochs (with Early Stopping)
allowed the network to keep learning until validation accuracy stopped improving.

* Slightly deeper / wider layers
increased the model's capacity to capture the Tic-Tac-Toe board patterns.

'''

'\nThe first configuration (baseline_more_epochs) gave the best results:\naccuracy = 0.9948, loss = 0.0894.\n\nThat improvement mainly came from:\n\n* Training for more epochs (with Early Stopping)\nallowed the network to keep learning until validation accuracy stopped improving.\n\n* Slightly deeper / wider layers\nincreased the model’s capacity to capture the Tic-Tac-Toe board patterns.\n\n'